In [1]:
import numpy as np
import cv2

In [2]:
# load trained model
yolo_model = cv2.dnn.readNetFromDarknet('trained_model/cov_yolov4.cfg','trained_model/cov_yolov4_2000.weights')

# Get all layers from the yolo network
# Loop and find the last layer (output layer) of the yolo network 
yolo_layers = yolo_model.getLayerNames()
yolo_output_layer = [yolo_layers[layer[0] - 1] for layer in yolo_model.getUnconnectedOutLayers()]

In [3]:
testImges = open('yolo/darknet/sign_data/test.txt', 'r')
testImges = [i for i in testImges.read().split('\n')]

In [4]:
class_labels = ['traffic sign']
class_colors = np.int32([0, 255, 0])
class_colors = np.tile(class_colors,(len(class_labels), 1))

In [5]:
def predict_sign(path):
    
    # load test image 
    img =cv2.imread(path)

    # construct a blob for img
    imgBlob = cv2.dnn.blobFromImage(img, 0.003922, (416, 416), swapRB=True, crop=False)

    # get image shape
    height = img.shape[0]
    width = img.shape[1]

    # input preprocessed blob into the model
    yolo_model.setInput(imgBlob)
    # get prediction
    obj_detection_layers = yolo_model.forward(yolo_output_layer)
    
    # create empty lists to save parameters
    class_ids_list = []
    boxes_list = []
    confidences_list = []

    # loop over each of the layer outputs
    for object_detection_layer in obj_detection_layers:
        # loop over the detections
        for object_detection in object_detection_layer:

            # obj_detections[1 to 4] => will have the two center points, box width and box height
            # obj_detections[5] => will have scores for all objects within bounding box
            all_scores = object_detection[5:]
            predicted_class_id = np.argmax(all_scores)
            prediction_confidence = all_scores[predicted_class_id]

            # take only predictions with confidence more than 60%
            if prediction_confidence > 0.60:
                #get the predicted label
                predicted_class_label = class_labels[predicted_class_id]
                #obtain the bounding box co-oridnates for actual image from resized image size
                bounding_box = object_detection[0:4] * np.array([width, height, width, height])
                (box_center_x_pt, box_center_y_pt, box_width, box_height) = bounding_box.astype("int")
                start_x_pt = int(box_center_x_pt - (box_width / 2))
                start_y_pt = int(box_center_y_pt - (box_height / 2))

                # save class id, start x, y, width & height, confidences in a list
                # make sure to pass confidence as float and width and height as integers
                class_ids_list.append(predicted_class_id)
                confidences_list.append(float(prediction_confidence))
                boxes_list.append([start_x_pt, start_y_pt, int(box_width), int(box_height)])
                
    # apply non-maxima suppression
    max_value_ids = cv2.dnn.NMSBoxes(boxes_list, confidences_list, 0.5, 0.4)
    for max_valueid in max_value_ids:
        max_class_id = max_valueid[0]
        box = boxes_list[max_class_id]
        start_x_pt = box[0]
        start_y_pt = box[1]
        box_width = box[2]
        box_height = box[3]
        end_x_pt = start_x_pt + box_width
        end_y_pt = start_y_pt + box_height

        #get the predicted class id and label
        predicted_class_id = class_ids_list[max_class_id]
        predicted_class_label = class_labels[predicted_class_id]
        prediction_confidence = confidences_list[max_class_id]

        #get a random mask color from the numpy array of colors
        box_color = class_colors[predicted_class_id]

        #convert the color numpy array as a list and apply to text and box
        box_color = [int(c) for c in box_color]

        # print the prediction in console
        predicted_class_label = "{}: {:.2f}%".format(predicted_class_label, prediction_confidence * 100)
        print("predicted object {}".format(predicted_class_label))

        # draw rectangle and text in the image
        cv2.rectangle(img, (start_x_pt, start_y_pt), (end_x_pt, end_y_pt), box_color, 1)
        cv2.putText(img, predicted_class_label, (start_x_pt, start_y_pt-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)

    cv2.imshow("Detection Output", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
basePath ='yolo/darknet/'
for img in testImges[:4]:
    predict_sign(basePath + img)

In [21]:
predict_sign(basePath + testImges[15])

predicted object traffic sign: 97.39%
predicted object traffic sign: 92.88%
